<a href="https://colab.research.google.com/github/mz20082/CE888-Assignment1/blob/main/Offensive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files 
uploaded = files.upload()

Saving mapping.txt to mapping (1).txt
Saving test_labels.txt to test_labels (1).txt
Saving test_text.txt to test_text (1).txt
Saving train_labels.txt to train_labels (1).txt
Saving train_text.txt to train_text (1).txt
Saving val_labels.txt to val_labels (1).txt
Saving val_text.txt to val_text (1).txt


In [ ]:
#importing and installing the required libraries for our prediction

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import csv
import re
import itertools
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate, validation_curve
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, precision_recall_curve

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [4]:
#read the libraries which had been uploaded using import function
train  = pd.read_csv('train_text.txt',error_bad_lines=False)
test = pd.read_csv('test_text.txt',error_bad_lines=False)

b'Skipping line 6: expected 1 fields, saw 2\nSkipping line 12: expected 1 fields, saw 3\nSkipping line 18: expected 1 fields, saw 3\nSkipping line 120: expected 1 fields, saw 4\nSkipping line 125: expected 1 fields, saw 3\nSkipping line 127: expected 1 fields, saw 3\nSkipping line 135: expected 1 fields, saw 2\nSkipping line 159: expected 1 fields, saw 3\nSkipping line 177: expected 1 fields, saw 3\nSkipping line 204: expected 1 fields, saw 3\nSkipping line 220: expected 1 fields, saw 3\nSkipping line 226: expected 1 fields, saw 2\nSkipping line 247: expected 1 fields, saw 2\nSkipping line 258: expected 1 fields, saw 2\nSkipping line 291: expected 1 fields, saw 2\nSkipping line 332: expected 1 fields, saw 2\nSkipping line 349: expected 1 fields, saw 2\nSkipping line 380: expected 1 fields, saw 2\nSkipping line 395: expected 1 fields, saw 6\nSkipping line 411: expected 1 fields, saw 4\nSkipping line 455: expected 1 fields, saw 2\nSkipping line 493: expected 1 fields, saw 2\nSkipping lin

In [9]:
#changing the column names for train dataset
train.columns = ['tweet']
train


,tweet
0,@user Eight years the republicans denied obama...
1,@user Get him some line help. He is gonna be j...
2,@user @user She is great. Hi Fiona!
3,@user She has become a parody unto herself? Sh...
4,@user Your looking more like a plant #maga #wa...
...,...
11376,@user I wonder if they are sex traffic victims?
11377,@user Do we dare say he is... better than Nyjer?
11378,@user No idea who he is. Sorry
11379,#Professor Who Shot Self Over Trump Says Gun C...


In [11]:
#changing the column name for test dataset
test.columns = ['tweet']
test

,tweet
0,@user @user @user I got in a pretty deep debat...
1,Angels now have 6 runs. Five of them have come...
2,#naturephotography. #nature. #birds #wild in #...
3,#i2 This speaks for itself (NSFW)
4,#RAP is a form of ART! Used to express yoursel...
...,...
581,#Hillary and @ least 16 other #AngryDemocrats ...
582,#Denver's venues are STACKED with epic shows t...
583,#CNN irrationally argues 4 legalising #abortio...
584,#Conservatives don’t care what you post..it’s ...


In [12]:
#reading the uploaded train and test labels
tr = pd.read_csv('train_labels.txt',error_bad_lines=False)
ts = pd.read_csv('test_labels.txt',error_bad_lines=False)

In [14]:
#renaming the column names for train and test labels
tr.columns = ['label']
ts.columns = ['label']

In [16]:
#using the function concat we are concating train and test dataset
train = pd.concat([train,tr])
test = pd.concat([test,ts])

In [17]:
train

,tweet,label
0,@user Eight years the republicans denied obama...,NaN
1,@user Get him some line help. He is gonna be j...,NaN
2,@user @user She is great. Hi Fiona!,NaN
3,@user She has become a parody unto herself? Sh...,NaN
4,@user Your looking more like a plant #maga #wa...,NaN
...,...,...
11910,NaN,1.0
11911,NaN,0.0
11912,NaN,0.0
11913,NaN,0.0


In [18]:
#Downloading NLTK Stopwords & treebank
nltk.download('stopwords')
nltk.download('treebank')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [19]:

wpt = nltk.WordPunctTokenizer()
stopWords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
part_of_speech_taggin = nltk.pos_tag
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [20]:
contractions = {
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))


def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [22]:
def document_filter_tagging(doc):
    ## Remove HTML Characters
    html_parser = HTMLParser()
    doc = html_parser.unescape(doc)
    ## Decode the Data (keep all the data in UTF-8 encoded characters
    #doc = doc.encode.("utf8").encode('ascii','ignore')
    ## Remove Emoji if exist using Regular Expressions
    emoji_pattern = re.compile(
        u"(\ud83d[\ude00-\ude4f])|"  # emoticons
        u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
        u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
        u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
        u"(\ud83c[\udde0-\uddff])"   # flags (iOS)
        "+", flags=re.UNICODE)
    emoji_pattern.sub(r'', doc)
    
    ## Split attached words like "Fu*kYou"
    #doc = ' '.join(re.findall('[A-Z][^A-Z]+', doc))
    ## Standardizing Words "like FUUUUUUUU*K => FU*K"
    doc = ''.join(''.join(s)[:2] for _, s in itertools.groupby(doc))
    ## Lower canse
    doc = doc.lower()
    ## Expand contraction "isn't => is not"
    doc = expand_contractions(doc)
    ## Transform Slang Words NOT YET :( :(
    ## remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.strip()  # NO leading or tailing white spaces
    
    # tokenize document
    tokens = wpt.tokenize(doc)

       
  # Tagging each Token with POS
    text = tokens
    text_tagged = nltk.pos_tag(text)
    new_text = []
    for word in text_tagged:
        new_text.append(word[0] + "/" + word[1])
    ## filter stopwords out of document
    filtered_tokens=[]
    for token in new_text:
        if token.split('/')[0] not in stopWords:
            filtered_tokens.append(token)

    doc = ' '.join(filtered_tokens)
    
   
    # Remove punctuactions
    # This will remove every punctuation in the Document
    doc.translate(string.punctuation)
    return doc

In [23]:
def document_filter(doc):
    ## Remove HTML Characters
    html_parser = HTMLParser()
    doc = html_parser.unescape(doc)
    ## Decode the Data (keep all the data in UTF-8 encoded characters
    #doc = doc.encode.("utf8").encode('ascii','ignore')
    ## Remove Emoji if exist using Regular Expressions
    emoji_pattern = re.compile(
        u"(\ud83d[\ude00-\ude4f])|"  # emoticons
        u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
        u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
        u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
        u"(\ud83c[\udde0-\uddff])"   # flags (iOS)
        "+", flags=re.UNICODE)
    emoji_pattern.sub(r'', doc)
    
    ## Split attached words like "Fu*kYou"
    #doc = ' '.join(re.findall('[A-Z][^A-Z]+', doc))
    ## Standardizing Words "like FUUUUUUUU*K => FU*K"
    doc = ''.join(''.join(s)[:2] for _, s in itertools.groupby(doc))
    ## Lower canse
    doc = doc.lower()
    ## Expand contraction "isn't => is not"
    doc = expand_contractions(doc)
    ## Transform Slang Words NOT YET :( :(
    ## remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.strip()  # NO leading or tailing white spaces
    
    # tokenize document
    tokens = wpt.tokenize(doc)
  
   ## filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stopWords]
  
    # Stemming the Document using NLTK
    stemmed_tokens = []
    for token in filtered_tokens:
        stemmed_tokens.append(stemmer.stem(token))
    doc = ' '.join(stemmed_tokens)
    # Lemmatization with NLTK
    lemmatized_tokens = []
    for token in stemmed_tokens:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    doc = ' '.join(filtered_tokens)
    
   
    # Remove punctuactions
    # This will remove every punctuation in the Document
    doc.translate(string.punctuation)
    return doc

In [24]:

#Function for generating feature vectors
def cv_Vectorization(data, range=(1, 2)):

    WoB_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=range, max_features=10000)
    cv = WoB_vect.fit_transform(data.values.astype('U')).toarray()


    return cv, cv_valid

# TF_IDF vectorizer based on 2-grams words


def tfdif_vectorization(data_train, range=(2, 3)):

    
    tfdif_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=range, max_features=10000, min_df=0, max_df=1)
    tfdif_train_x = tfdif_vectorizer.fit_transform(data_train.values.astype('U'))
    
    

    return tfdif_train_x


def bow_tfidf_vectorization(data_train, range=(2, 3)):
    WoB_vect = CountVectorizer(
        analyzer='word', token_pattern=r'\w{1,}', ngram_range=range, max_features=5000)
    cv = WoB_vect.fit_transform(data_train.values.astype('U'))
    #cv_valid = WoB_vect.fit_transform(data_validation.values.astype('U'))

    tfdif_transformer = TfidfTransformer()
    tfdif_train_x = tfdif_transformer.fit_transform(cv)
    #tfdif_validation_x = tfdif_transformer.fit_transform(cv_valid)

    return tfdif_train_x